In [5]:
import pandas as pd
from graph_tool.all import *
from collections import defaultdict
import numpy as np
import matplotlib as plt

# Load data

location = "/mnt/c/Users/sauba/Desktop/Drosophila_BUSCO_ERC/4.BUSCO_filtered/7.ERC"
file_name = "ERC_correlation_longform_annotated"

# df = pd.read_csv(f"{location}/{file_name}.csv")
# df = df[(df['Correlation'] > 0.4) & (df['Correlation'] < 1.0)]
# df = df[df['Correlation'] < 1.0]

g = Graph(directed=False)

name_prop = g.new_vertex_property("string")
weight = g.new_edge_property("float")
pen_width = g.new_edge_property("float")
edge_label = g.new_edge_property("string")

vsize = g.new_vertex_property("float")
vsize.a = 1

min_w, max_w = 1.0, 5.0
min_corr = 0.4
max_corr = 0.99


node_map = {}
def get_vertex(name):
    if name not in node_map:
        v = g.add_vertex()
        node_map[name] = v
        name_prop[v] = name
    return node_map[name]

added_edges = set()

for _, row in df.iterrows():
    
    source = row['Annotation_gene1']
    target = row['Annotation_gene2']
    if type(source)== float:
       source = "nan" 
    if type(target)== float:
       target = "nan" 
    try:
        key = tuple(sorted([source, target]))
    except:
        print(source,target, row)
        assert False

    if key in added_edges:
        continue
    added_edges.add(key)
    v1 = get_vertex(source)
    v2 = get_vertex(target)
    
    if row["value"] > 0.4 and row["value"] < 1.0:
        corr = row["value"]
        e = g.add_edge(v1, v2)
        weight[e] = corr
        scaled = min_w + (corr - min_corr) / (max_corr - min_corr) * (max_w - min_w)
        pen_width[e] = scaled
        edge_label[e] = f"{corr:.2f}"


# deg = g.degree_property_map("total")
# max_deg = max(deg.a) if max(deg.a) > 0 else 1
# color = g.new_vertex_property("vector<float>")
# for v in g.vertices():
#     d = deg[v]
#     color[v] = [d / max_deg, 0.2, 1.0 - d / max_deg, 1.0]  # RGBA
#     # color[v] = [x * 0.1 for x in range(0, 10)]

deg = g.degree_property_map("total")
max_deg = max(deg.a) if max(deg.a) > 0 else 1
color = g.new_vertex_property("vector<float>")



# Choose a colormap, for example 'viridis', 'plasma', 'magma', 'cividis', 'inferno'
# You can explore other colormaps in matplotlib's documentation.
cmap = plt.colormaps['Spectral_r']
list_of_degree = []
degree_list_plus = []
for v in g.vertices():
    d = deg[v]
    list_of_degree.append(d)
    degree_list_plus.append((name_prop[v],d))
    # Normalize degree to be between 0 and 1 for colormap mapping
    log_deg = np.log1p(d) # log1p(x) is log(1+x)
    max_log_deg = np.log1p(max_deg)
    normalized_degree = log_deg / max_log_deg if max_log_deg > 0 else 0
    # normalized_degree = d / max_deg if max_deg > 0 else 0
    
    # Get RGBA color from the colormap
    rgba_color = cmap(normalized_degree)
    color[v] = rgba_color # Assign the RGBA tuple directly

# assert False


# --- COMMUNITY DETECTION ---
from graph_tool.inference import minimize_blockmodel_dl
import matplotlib.cm as cm
import numpy as np

# Detect communities (blocks)
state = minimize_blockmodel_dl(g)
blocks = state.get_blocks()

# Color nodes by block
block_ids = blocks.a
num_blocks = len(set(block_ids))


block_dict = defaultdict(list)

for v in g.vertices():

    block_id = blocks[v]
    gene_name = name_prop[v]
    block_dict[block_id].append(gene_name)

# --- CLUSTER-AWARE LAYOUT ---
# This will pull clusters apart based on community membership
pos = sfdp_layout(g, groups=blocks)

# Draw graph with community-based colors
# file_name = "rho_set_non_dia"
graph_draw(g,
           pos=pos,  # <-- use the new layout
           vertex_text=name_prop,
           vertex_fill_color=color,
           vertex_font_size=10,
           vertex_size=vsize,
           edge_pen_width=pen_width,
           edge_text=edge_label,
           edge_font_size=8,
           output_size=(800, 800),
           # output_size=(2000, 2000),
           output=f"{location}/{file_name}.png") 
output = "Block,Gene"
for key, value in block_dict.items():
    for gene_name in value:
        output += f"\n{key},{gene_name}"

with open(f"{location}/{file_name}_cluster.csv", 'w') as out_file:
    out_file.write(output)
output = ''
with open(f"{location}/{file_name}_degree.csv", 'w') as out_file:
    for degree_num in degree_list_plus:
        output +=  str(degree_num[0])+","+ str(degree_num[1])+ "\n"
    # output = "\n".join(list_of_degree)
    out_file.write(output)

import matplotlib.pyplot as plt
plt.hist(list_of_degree, bins=max(list_of_degree), edgecolor='black')

# Add labels
plt.title('Histogram')
plt.xlabel('Degree')
plt.ylabel('Count')

# Show plot
plt.savefig(f"{location}/{file_name}_degree_histogram_output.pdf", format='pdf', bbox_inches='tight')
plt.close()

print(degree_list_plus)


[('RpL32', 10), ('PNUTS', 28), ('acs', 107), ('CG15237', 24), ('Pbp95', 204), ('Oatp26F', 12), ('CG8646', 24), ('CG4666', 8), ('CG5882', 34), ('CG3880', 29), ('AP-2mu', 69), ('fw', 35), ('RYBP', 12), ('Arr1', 81), ('CG7215', 46), ('AP-1gamma', 10), ('CG9265', 44), ('Nprl3', 26), ('Ugt36A1', 12), ('bdwf', 78), ('Sur-8', 30), ('Grx5', 24), ('CG13315', 20), ('Kat60', 138), ('GstE4', 129), ('Pdf', 8), ('Aps', 3), ('Teh1', 23), ('CG7309', 88), ('mGluR', 263), ('PIG-V', 25), ('l(3)05822', 10), ('CG5800', 50), ('rod', 22), ('mRpL2', 17), ('Acp65Aa', 19), ('Swt1', 121), ('mi', 5), ('Cmpk', 8), ('Prosbeta6', 46), ('wake', 21), ('CG9411', 43), ('Elal', 133), ('CG13077', 10), ('mus302', 35), ('Sil1', 205), ('T3dh', 7), ('MED16', 15), ('CG9628', 54), ('Usp15-31', 59), ('zen', 51), ('scat', 40), ('CG7183', 41), ('CG9356', 34), ('Vha16-5', 41), ('OXA1L', 11), ('Fem-1', 23), ('kek6', 40), ('ClC-c', 21), ('l(1)G0004', 41), ('Taf3', 358), ('ND-B14', 7), ('CG9304', 6), ('GatA', 5), ('CG5273', 132), ('PG

In [4]:
print(type(target) == float)
    

True


In [55]:
output = ''
with open(f"{location}/{file_name}_degree.csv", 'w') as out_file:
    for degree_num in degree_list_plus:
        output +=  str(degree_num[0])+","+ str(degree_num[1])+ "\n"
    # output = "\n".join(list_of_degree)
    out_file.write(output)